# Social Platform Emulator

In [2]:
import numpy as np

In [2]:
#N = 10 # number of users

In [3]:
#Leaders = {}
#for i in range(N-1):
#    Leaders[i] = i+1
#Leaders[N-1] = 0
#print(Leaders)

In [2]:
K = 1 # Wall, Newsfeed list size

Wall is finite for all users. We apply FIFO replacement principle. The Newsfeed is not needed, because we have no information on the policy of the OSP or the user policy to choose among many. We know: 1) which post is tweeted-retweeted and the time-stamp, and the tweet origin.

For each user we create a Wall list of max size $K$.

In [3]:
Wall = {}

Data Format in "TextTweet"

tweetid timestamp userid retweetid

**Question:** is retweetid, the tweetid from the origin, or from the leader? **A: From the origin. Leader information is lost.**

In [7]:
f = open("/Users/Fishbone/Desktop/NEWSFEEDfresh/PYTHON/NewResults/PsiEmul.tronc.txt")
tc = 0
tc0 = 0
for lign in f:
    lign = lign.split()
    tc +=1
    l1 = float(lign[1])
    if l1!=0:
        tc0+=1
f.close()
print(tc)
print(tc0)

3754043
405236


In [9]:
f = open("/Users/Fishbone/Desktop/NEWSFEEDfresh/PYTHON/NewResults/Psi.tronque.sorted22gr.0.5804715.txt")
tc = 0
tc0 = 0
for lign in f:
    lign = lign.split()
    tc +=1
    l1 = float(lign[1])
    if l1!=0:
        tc0+=1
f.close()
print(tc)
print(tc0)

5804715
405236


In [6]:
#f = open("TextTweet.txt")
f = open("tweets.parsed.sorted22n.txt")

In [7]:
Author = {}

In [8]:
for lign in f:
    lign = lign.split()
    tweetid = int(lign[0])
    userid = int(lign[2])
    Author[tweetid] = userid

In [9]:
#Author

In [10]:
f.close()

In [11]:
print(len(Author))

27916100


In [12]:
print(74400 in Author)

False


In [13]:
assert 74400 not in Author

We introduce a disctionary Q, which contains many dictionries, each related to a userid. $Q[userid][auth-out]$ saves the time periods that a post from $[auth-out]$ stays on the Wall of $[userid]$.

In [14]:
Q = {}
FirstP = {}

**Note:** Are we happy with Q as matrix? or list of lists? or txt? or dictionary? **A: Dictionary of Dictionaries**

** Important!**

if the tweet is original then the person who posted is the author.
    else it is a retweet. If the original post was observed and saved in "Authors",
   we know the author.
   else the retweet has origin outside the dataset. In this case we assume that
   the every user with this retweet is considered as the original author.

In [15]:
#f = open("TextTweet.txt")
Amphitrions = {}
f = open("tweets.parsed.sorted22n.txt")
nb_neg = 0
# debug
i=0
j=0
Time0 = None
timestamp_old = 0
count_ties = 0
for lign in f:
    #if j<6:
    #    j+=1
    #    print("original=",lign)
    lign = lign.split()
    tweetid = int(lign[0])
    tstamp = int(lign[1])
    userid = int(lign[2])
    rtid = int(lign[3])  
    if Time0 ==None:
        Time0 = tstamp
    if tstamp == timestamp_old:
        count_ties+=1
    timestamp_old = tstamp
    if userid not in Amphitrions:
        Amphitrions[userid] = {}
    #if j<6:
    #    print("split=",tweetid, tstamp, userid, rtid)  
    # if the tweet is original then the person who posted is the author.
    # else it is a retweet. If the original post was observed and saved in "Authors",
    # we know the author.
    # else the retweet has origin outside the dataset. In this case we assume that
    # the first retweeter is the original author.
    if rtid == -1:
        auth = userid
    elif rtid in Author:
        auth = Author[rtid] # it is the tweet origin ID
    else:
        assert rtid not in Author
        auth = userid
        Author[tweetid] = userid
    if userid not in Wall:
        Wall[userid] = []
        Q[userid] = {}
    if len(Wall[userid])==0:
        FirstP[userid] = tstamp
    Wall[userid].append((tweetid,tstamp,auth))
    if len(Wall[userid])>K:
        auth_out = Wall[userid][0][2]
        time_wall = tstamp-Wall[userid][0][1]
        #if time_wall < 0:
        #    nb_neg += 1
        #    # debug
        #    if i<6:
        #        i+=1
        #        #print(userid, tstamp)
        #if userid not in Q:
        #    Q[userid] = {}
        if auth_out not in Q[userid]:
            Q[userid][auth_out] = 0
        # CORRECTION
        #Q[userid][auth_out] += time_wall
        #
        # CORRECTION
        if time_wall == 0:
            if auth_out not in Amphitrions[userid]:
                Amphitrions[userid][auth_out] = 0 
            Amphitrions[userid][auth_out] += 1
        else: 
            if auth_out not in Amphitrions[userid]:
                Amphitrions[userid][auth_out] = 0 
            Amphitrions[userid][auth_out] += 1
            Ndt = 0
            for author in Amphitrions[userid]:
                Ndt += Amphitrions[userid][author]
            for author in Amphitrions[userid]:
                Q[userid][author] += time_wall/Ndt*Amphitrions[userid][author]
            Amphitrions[userid] = {}
        #### END of CORRECTION
        #if userid == 3474:
        #    print(auth_out, time_wall)
        Wall[userid] = Wall[userid][1:]
        assert( len(Wall[userid]) == K )
#print(nb_neg)
print(count_ties)

23889642


In [16]:
f.close()

In [18]:
print(Time0)

1398895201


In [19]:
27916100-4026458

23889642

In [20]:
print(len(Author))

27916100


In [21]:
#print(Wall,'\n')
#print('Q=',Q)
#print('FirstP=',FirstP)

**Note1:** EndPoint of simulation is the time of last arrival.

In [22]:
EndP = tstamp
print(EndP)

1406843993


**Note2:** Post-processing: Treat extra users with Wall content less than $K$.

In [23]:
for u in Wall:
    if len(Wall[u])>0:
        auth_out = Wall[u][0][2]
        time_wall = EndP-Wall[u][0][1]
        if u not in Q:
            Q[u] = {}
        if auth_out not in Q[u]:
            Q[u][auth_out] = 0
        #Q[u][auth_out] += time_wall
        # CORRECTION
        if time_wall == 0:
            if auth_out not in Amphitrions[u]:
                Amphitrions[u][auth_out] = 0 
            Amphitrions[u][auth_out] += 1
        else: 
            if auth_out not in Amphitrions[u]:
                Amphitrions[u][auth_out] = 0 
            Amphitrions[u][auth_out] += 1
            Ndt = 0
            for author in Amphitrions[u]:
                Ndt += Amphitrions[u][author]
            for author in Amphitrions[u]:
                # Q[u][author] += time_wall/Ndt*Amphitrions[u][author]
                ## CORRECTION 2: circularite
                Q[u][author] += (time_wall+FirstP[u]-Time0)/Ndt*Amphitrions[u][author]
            Amphitrions[u] = {}
        #### END of CORRECTION

In [24]:
#print(Q)

In [25]:
Q[3474]

{2083: 6.5,
 327: 6.5,
 408: 114161.0,
 3474: 926459.0,
 2660: 666122.0,
 16559: 5431010.0,
 23742: 811027.0}

In [26]:
Q[3474]

{2083: 6.5,
 327: 6.5,
 408: 114161.0,
 3474: 926459.0,
 2660: 666122.0,
 16559: 5431010.0,
 23742: 811027.0}

**Estimated Qs**

In [27]:
EstQ = {}
nb0 = 0
for u in Q:
    EstQ[u] = {}
    for j in Q[u]:
        if EndP-FirstP[u] == 0:
            nb0 += 1
            assert len(Q[u]) == 1
            EstQ.pop(u)
        else:
            #EstQ[u][j] = Q[u][j]/(EndP-FirstP[u])
            # CORRECTION 2: circularity
            EstQ[u][j] = Q[u][j]/(EndP-Time0)
#print(EstQ)
print(nb0)

0


In [28]:
N = len(EstQ)
print(N)

6020228


**EstQ[user][leader]:** is the proportion of time that posts of "leader" is on 1-st position of "user" Wall. Hence: sum{leaders}EstQ[user][leader] = 1.

In [29]:
testU =2500
print(EstQ[testU])

{1548: 0.6614825246402221, 2500: 0.33851747535977794}


In [30]:
Psi = 0
for user in EstQ[testU]:
    Psi+=EstQ[testU][user]
print(Psi)

1.0


In [31]:
#jupyter nbconvert --to script OSPemul.ipynb
#through command line convert to .py

**Calculate Influence of user on his followers**

In [32]:
Influence = {}
for follower in EstQ:
    for leader in EstQ[follower]:
        if leader not in Influence:
            Influence[leader] = {}
        Influence[leader][follower] = EstQ[follower][leader]

In [33]:
print(Influence[158410])

{17700: 0.10512050132900698, 158410: 0.9339905988230665, 172286: 0.26822465602320456, 444382: 5.032211183787424e-07, 1876907: 0.10557893576785}


In [34]:
print(Influence[158410])

{17700: 0.10512050132900698, 158410: 0.9339905988230665, 172286: 0.26822465602320456, 444382: 5.032211183787424e-07, 1876907: 0.10557893576785}


In [35]:
print(Influence[5])

{5: 1.0, 10728: 0.6129629508483805, 57295: 0.7196136217930976, 1816833: 1.0}


In [36]:
print(Influence[5])

{5: 1.0, 10728: 0.6129629508483805, 57295: 0.7196136217930976, 1816833: 1.0}


In [37]:
Psi = {}
for user in Influence:
    Psi[user] = 0
    for follower in Influence[user]:
        if follower != user:
            Psi[user]+=Influence[user][follower]
    Psi[user] = Psi[user]/(N-1)

In [38]:
Psi[158410]

7.955258104738906e-08

In [39]:
Psi[158410]

7.955258104738906e-08

In [40]:
Psi[1108115]

8.305334665951965e-08

In [41]:
Psi[21586] #4138, 15698, 17896, 19678, 20116, 20198, 20374, 20376, 21586

5.725805124780818e-12

In [42]:
Psi[266]

7.555219818877829e-06

In [43]:
Psi[11604]

0.018266619353711842

In [ ]:
#keysortL2 = []
#for k in range(1000):
#    

In [36]:
import gzip
f = open('PsiCorrCirc20190208','w')
for u in Psi:
    f.write("%d %g\n"%(u, Psi[u]))
f.close()

In [28]:
#import gzip
#f = open('Psi','w')
#for u in Psi:
#    f.write("%d %g\n"%(u, Psi[u]))
#f.close()


## **Part II: Extracting Info from data to feed the analysis**

In [37]:
Ntweet = {}
Nrtweet = {}
LeadGraph = {}
FirstT = None
LastT = None
el=0
el2=0
f = open("tweets.parsed.sorted22n.txt")
for lign in f:
    lign = lign.split()
    tstamp = int(lign[1])
    userid = int(lign[2])
    rtid = int(lign[3])
    #if el<21:
    #    print(userid, rtid)
    #    el+=1
    if FirstT == None:
        FirstT = tstamp
    if userid not in Ntweet:
        Ntweet[userid] = 0
        Nrtweet[userid] = 0
        LeadGraph[userid] = set() 
    
    if rtid == -1:
        Ntweet[userid] += 1
    else: 
        if rtid in Author:
            LeadGraph[userid].add(Author[rtid])
            Nrtweet[userid] += 1
        else:
            Ntweet[userid] += 1
LastT = tstamp
f.close()  

In [38]:
Rtweet = {}
Rrtweet = {}
for user in Ntweet:
    Rtweet[user] = Ntweet[user]/(LastT-FirstT)
    Rrtweet[user] = Nrtweet[user]/(LastT-FirstT)

In [51]:
Ntweet[11604]

1142

In [31]:
k = 0
for user in Rtweet:
    print(user, 'T=', Rtweet[user],'R=', Rrtweet[user])
    print(LeadGraph[user],'\n')
    if k==10:
        break
    else: 
        k+=1

3039 T= 3.887383139475785e-05 R= 0.0
set() 

790 T= 6.54187453892365e-06 R= 0.0
set() 

4806 T= 8.30314845324925e-06 R= 0.0
set() 

175 T= 8.164762645695094e-05 R= 0.0
set() 

6 T= 0.0004423313630549145 R= 0.0
set() 

831 T= 0.00012253434232522377 R= 0.0
set() 

25 T= 0.0004180509440931402 R= 0.0
set() 

374 T= 5.459949134409354e-05 R= 0.0
set() 

2646 T= 1.1322475163521703e-05 R= 0.0
set() 

250 T= 9.699587056750258e-05 R= 0.0
set() 

194 T= 0.0010933736849574123 R= 0.0
set() 



In [32]:
len(Rtweet)

6020228

In [36]:
Rtweet[17700]

2.1386897531096547e-06

In [36]:
len([0 for u in Rtweet if Rtweet[u]==0])

3416701

In [ ]:
#jupyter nbconvert --to script OSPemul.ipynb
#through command line convert to .py